In [1]:
import pandas as pd
import numpy as np
import re
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Rahul checkpoint 4 - evaluation metrics

Context Relevance, Context Utilization,Completeness and Adherence.
Values between 0 and 1 and 1 is the best

Compute RMSE and AUCROC by comparing with the original Context Relevance, Context Utilization and Adherence values in the dataset. (refer RAGBench paper)

In [2]:
results_df = pd.read_csv("raw_mistral_qwen_deepseek_results_df.csv")

In [3]:
results_df

,id,dataset_name,question,original_documents,original_response,retrieved_documents,qwen_05B,DeepSeek,llm_response
0,358,covidqa,What role does T-cell count play in severe hum...,['Title: Emergent severe acute respiratory dis...,The T-cell count plays a crucial role in sever...,['Title: Emergent severe acute respiratory dis...,What role does T-cell count play in severe hum...,Question: What role does T-cell count play in ...,"The semicontaminated area, as mentioned in the..."
1,457,covidqa,What is MVO?,['Title: Gene Knockdowns in Adult Animals: PPM...,MVO is not defined or mentioned in the provide...,['Intraocular pressure (IOP) is the fluid pres...,What is MVO?['Title: Gene Knockdowns in Adult ...,Question: What is MVO?\nContext: ['Title: Gene...,"Based on the context provided, as of 21 Februa..."
2,1172,covidqa,What have sero-surveys of MERS virus found?,"['Title: MERS coronavirus: diagnostics, epidem...",Sero-surveys of MERS virus found no sign of ME...,"['Title: MERS coronavirus: diagnostics, epidem...",What have sero-surveys of MERS virus found?['T...,Question: What have sero-surveys of MERS virus...,"Based on the context provided, sero-surveys of..."
3,1033,covidqa,Where was alpha-tubulin found least abundantly...,['Title: Nucleolar Protein Trafficking in Resp...,Alpha-tubulin was found least abundantly in th...,['Title: Nucleolar Protein Trafficking in Resp...,Where was alpha-tubulin found least abundantly...,Question: Where was alpha-tubulin found least ...,Alpha-tubulin was found to be weakly detected ...
4,1273,covidqa,What does the semicontaminated area include?,['Title: The Battle Against Coronavirus Diseas...,The semicontaminated area includes the fever-C...,['Title: The Battle Against Coronavirus Diseas...,What does the semicontaminated area include?['...,Question: What does the semicontaminated area ...,"In the context provided, T-cell count seems to..."
...,...,...,...,...,...,...,...,...,...
103,finqa_test_96,tatqa,When are multi-year agreement customers invoiced?,"['Contract Balances [[""(In millions)"", """", """",...",Multi-year agreement customers are generally i...,['Timing of revenue recognition may differ fro...,When are multi-year agreement customers invoic...,Question: When are multi-year agreement custom...,Multi-year agreement customers are typically i...
104,finqa_test_199,tatqa,What is the percentage change in the value of ...,"['Inventories and Inventory Valuation [[""March...",To calculate the percentage change in the valu...,"['At December 31, 2019 and 2018, the Company h...",What is the percentage change in the value of ...,Question: What is the percentage change in the...,"Based on the context provided, we do not have ..."
105,techqa_DEV_Q243,techqa,"Using cobol copybooks Sometimes, there will be...",[' TECHNOTE (TROUBLESHOOTING)\n\nPROBLEM(ABSTR...,"Yes, there is a specific format for COBOL copy...",['z/os A FIX IS AVAILABLE\nObtain the fix for...,"Using cobol copybooks Sometimes, there will be...","Question: Using cobol copybooks Sometimes, the...","Based on the context provided, there is no spe..."
106,techqa_DEV_Q280,techqa,Is WebSphere Transformation Extender (WTX) sup...,['DTXINST; Integration Server; WESB; Install; ...,"Based on the provided context, WebSphere Trans...",['WebSphere Transformation Extender; WebSphere...,Is WebSphere Transformation Extender (WTX) sup...,Question: Is WebSphere Transformation Extender...,"Based on the context provided, there is no men..."


In [4]:
#Rahul for sentence tokenizing
def convert_text(input_text):
    # Ensure input_text is a string
    if not isinstance(input_text, str):
        raise ValueError("Input must be a string")

    # Parse the input text as a Python list
    entries = eval(input_text)

    result = []

    for idx, entry in enumerate(entries):
        sub_result = []

        # Split the entry into title and passage
        lines = entry.split('\n')
        for line_idx, line in enumerate(lines):
            if line_idx == 0:  # Title line
                sub_result.append([f"{idx}a", line.strip()])
            else:  # Passage lines
                # Further split passage into sentences
                sentences = re.split(r'(?<=\.)\s', line.strip())
                for sentence_idx, sentence in enumerate(sentences):
                    if sentence:  # Ignore empty sentences
                        sub_result.append([f"{idx}{chr(98 + sentence_idx)}", sentence.strip()])

        result.append(sub_result)

    return result

In [5]:
# Convert 'original_documents' to string format if not already a string
results_df['original_documents'] = results_df['original_documents'].apply(lambda x: str(x) if not isinstance(x, str) else x)
results_df['retrieved_documents'] = results_df['retrieved_documents'].apply(lambda x: str(x) if not isinstance(x, str) else x)

# Now apply the convert_text function
results_df['original_documents_sentences'] = results_df.apply(lambda row: convert_text(row['original_documents']), axis=1)
results_df['retrieved_documents_sentences'] = results_df.apply(lambda row: convert_text(row['retrieved_documents']), axis=1)

In [6]:
def extract_specific_identifiers(processed_text):
    extracted = []

    for idx, entry in enumerate(processed_text):

        matching_items = [item[0] for item in entry if item[0][-1] in ['a', 'b', 'c', 'd', 'e', 'f']]

        # Renumber the matching items starting from 0
        renumbered_items = [f"{idx}{item[-1]}" for item in matching_items]

        extracted.extend(renumbered_items)

    return extracted

In [7]:
# Apply the new function to create a new column
results_df['original_documents_identifiers'] = results_df['original_documents_sentences'].apply(extract_specific_identifiers)
results_df['retrieved_documents_identifiers'] = results_df['retrieved_documents_sentences'].apply(extract_specific_identifiers)

In [8]:
import ast
from ast import literal_eval

def safe_parse_list(value):
    """Safely convert string representation of lists to actual lists"""
    if isinstance(value, str):
        try:
            return literal_eval(value)
        except (ValueError, SyntaxError):
            print(f"Failed to parse value: {value[:50]}...")  # Show first 50 chars of problematic data
            return []
    return value  # Return as-is if already a list

# Convert columns only if they contain string representations
results_df['original_documents_sentences'] = results_df['original_documents_sentences'].apply(safe_parse_list)
results_df['retrieved_documents_sentences'] = results_df['retrieved_documents_sentences'].apply(safe_parse_list)

def find_matching_titles(row):
    try:
        original = row['original_documents_sentences']
        retrieved = row['retrieved_documents_sentences']

        matching_docs = []
        seen_titles = set()

        for orig_doc in original:
            if not orig_doc:
                continue
            orig_title = orig_doc[0][1].strip().lower() if orig_doc[0] else ""

            for retr_doc in retrieved:
                if not retr_doc:
                    continue
                retr_title = retr_doc[0][1].strip().lower() if retr_doc[0] else ""

                if orig_title and orig_title == retr_title and orig_title not in seen_titles:
                    matching_docs.append(orig_doc)
                    seen_titles.add(orig_title)
                    break

        return matching_docs
    except Exception as e:
        print(f"Error processing row: {str(e)[:100]}...")
        return []

In [9]:
results_df['retrieved_relevant_documents_sentences'] = results_df.apply(find_matching_titles, axis=1)

In [10]:
results_df['retrieved_relevant_documents_sentences_identifiers'] = results_df['retrieved_relevant_documents_sentences'].apply(extract_specific_identifiers)

In [11]:
#update string to list in n python pandas dataframe results_df column original_documents_identifiers, update values from ['0a', '0b', '0c', '0d', '0e', '0f', '1a... to [0a, 0b, 0c, 0d, 0e, 0f, 1a
# Convert string representation of list to actual list
results_df['retrieved_relevant_documents_sentences_identifiers'] = results_df['retrieved_relevant_documents_sentences_identifiers'].apply(lambda x: [f"'{item}'" for item in x])

In [ ]:
#TRACe metrics
'''
TRACe evaluation framework consists of four metrics:
uTilization: Measures the fraction of the retrieved context that is used by the generator to produce the response.
Relevance: Measures the fraction of the retrieved context that is relevant to the input query.
Adherence: Measures whether all parts of the response are grounded in the context (synonymous with faithfulness and groundedness).
Completeness: Measures how well the response incorporates all the relevant information in the context.
'''

'\nTRACe evaluation framework consists of four metrics:\nuTilization: Measures the fraction of the retrieved context that is used by the generator to produce the response.\nRelevance: Measures the fraction of the retrieved context that is relevant to the input query.\nAdherence: Measures whether all parts of the response are grounded in the context (synonymous with faithfulness and groundedness).\nCompleteness: Measures how well the response incorporates all the relevant information in the context.\n'

In [12]:
def calculate_context_relevance(row):
    try:
        # Directly use list lengths without string operations
        len_retrieved = len(row['retrieved_relevant_documents_sentences_identifiers'])
        len_original = len(row['original_documents_identifiers'])

        return len_retrieved / len_original if len_original > 0 else 0
    except Exception as e:
        print(f"Error processing row {row['id']}: {str(e)}")
        return 0

# Apply the function
results_df['context_relevance'] = results_df.apply(calculate_context_relevance, axis=1)

In [22]:
def calculate_context_utilization(df):
    """
    Calculates the context utilization score for each row in a DataFrame and returns the mean utilization score.
    The utilization score is capped at 1.0 to ensure it doesn't exceed 1.

    Args:
        df (pd.DataFrame): DataFrame containing 'retrieved_documents_sentences' and 'original_documents_sentences' columns.

    Returns:
        float: Mean context utilization score, capped at 1.0.  Returns 0.0 in case of errors or empty original documents.
    """
    try:

        def calculate_row_context_utilization(row):
            """Calculates context utilization for a single row."""
            try:
                len_retrieved = len(row['retrieved_documents_sentences'])
                len_original = len(row['original_documents_sentences'])

                if len_original > 0:
                    utilization = len_retrieved / len_original
                    return min(utilization, 1.0)  # Cap the utilization at 1.0
                else:
                    return 0.0  # Handle cases where original document has no sentences
            except Exception as e:
                print(f"Error processing row {row['id']}: {str(e)}")
                return 0.0

        # Apply the calculation to each row and get the mean
        utilization_scores = df.apply(calculate_row_context_utilization, axis=1)
        return np.mean(utilization_scores)

    except Exception as e:
        print(f"Error in calculate_utilization: {str(e)}")
        return 0.0

# Apply the function
results_df['utilization_score'] = calculate_context_utilization(results_df)

In [14]:
def list_to_tuple(obj):
    """Recursively convert all lists to tuples for hashability"""
    if isinstance(obj, list):
        return tuple(list_to_tuple(x) for x in obj)
    return obj

def calculate_completeness(row):
    try:
        # Convert all nested lists to tuples
        original_set = set(list_to_tuple(doc) for doc in row['original_documents_sentences'])
        retrieved_set = set(list_to_tuple(doc) for doc in row['retrieved_documents_sentences'])

        # Calculate intersection ratio
        return len(original_set & retrieved_set) / len(original_set) if original_set else 0
    except Exception as e:
        print(f"Error processing row {row['id']}: {str(e)}")
        return 0

# Apply the function row by row
results_df['completeness_score'] = results_df.apply(calculate_completeness, axis=1)

In [15]:
def calculate_adherence(row):
    try:
        # Get the identifiers and ensure they're in list format
        original_ids = eval(row['original_documents_identifiers']) if isinstance(row['original_documents_identifiers'], str) else row['original_documents_identifiers']
        retrieved_ids = eval(row['retrieved_relevant_documents_sentences_identifiers']) if isinstance(row['retrieved_relevant_documents_sentences_identifiers'], str) else row['retrieved_relevant_documents_sentences_identifiers']

        # Clean up the retrieved IDs by removing extra quotes
        retrieved_ids = [id.strip("'\"") for id in retrieved_ids]

        # Convert to sets for comparison
        original_ids_set = set(original_ids)
        retrieved_ids_set = set(retrieved_ids)

        # Check if retrieved IDs are a subset of original IDs
        is_adherent = retrieved_ids_set.issubset(original_ids_set)

        return is_adherent
    except Exception as e:
        print(f"Error processing row {row['id']}: {str(e)}")
        return False

# Apply the function to create new column
results_df['is_adherent'] = results_df.apply(calculate_adherence, axis=1)

In [17]:
#function to add noise
def add_noise_to_sentence(sentence):
    """Add noise to a single sentence (sentence format: [identifier, text])"""
    if len(sentence) >= 2:
        text = sentence[1]
        words = text.split()
        if len(words) > 1:
            index = random.randint(0, len(words)-1)
            words[index] = ''.join(random.sample(words[index], len(words[index])))
            sentence[1] = ' '.join(words)
    return sentence

def add_noise(document_list):
    """Process nested document structure"""
    return [
        [
            add_noise_to_sentence(sentence)
            for sentence in doc
        ]
        for doc in document_list
    ]

In [18]:
results_df['noisy_response'] = results_df['retrieved_relevant_documents_sentences'].apply(add_noise)

In [19]:
def extract_text_from_documents(documents):
    """Extract text from nested document structure ([documents][sentences][id, text])"""
    return ' '.join(
        sentence[1]
        for doc in documents
        for sentence in doc
        if len(sentence) >= 2
    )

def calc_noise_robust(original, retrieved, noisy):
    # Extract text from nested document structures
    original_text = extract_text_from_documents(original)
    retrieved_text = extract_text_from_documents(retrieved)
    noisy_text = extract_text_from_documents(noisy)

    # Handle empty texts
    if not original_text or (not retrieved_text and not noisy_text):
        return 0.0

    # Create and fit TF-IDF vectorizer
    vectorizer = TfidfVectorizer(
        stop_words='english',
        token_pattern=r'(?u)\b\w+\b',
        lowercase=True
    )

    try:
        tfidf_matrix = vectorizer.fit_transform([original_text, retrieved_text, noisy_text])
    except ValueError:
        return 0.0

    # Calculate cosine similarities
    similarities = cosine_similarity(tfidf_matrix)

    # Calculate noise robustness score
    original_retrieved = similarities[0, 1] if tfidf_matrix.shape[0] > 1 else 0
    original_noisy = similarities[0, 2] if tfidf_matrix.shape[0] > 2 else 0

    return np.clip(original_retrieved - original_noisy, -1, 1)

In [20]:
results_df['noise_robustness'] = results_df.apply(
    lambda row: calc_noise_robust(
        row['original_documents_sentences'],
        row['retrieved_relevant_documents_sentences'],
        row['noisy_response']
    ),
    axis=1
)

In [ ]:
def extract_text_from_nested(doc_list):
    """Convert nested document structure to clean text"""
    return ' '.join(
        sentence[1]
        for doc in doc_list
        for sentence in doc
        if isinstance(sentence, list) and len(sentence) >= 2
    )

def calc_negative_rejection(original_text, retrieved_text, noisy_text):
    """
    Modified to accept pre-processed text inputs
    """
    # Create TF-IDF vectorizer
    vectorizer = TfidfVectorizer(stop_words='english',
                                token_pattern=r'(?u)\b\w+\b',
                                lowercase=True)

    try:
        tfidf_matrix = vectorizer.fit_transform([original_text, retrieved_text, noisy_text])
    except ValueError:
        return 0.0

    similarities = cosine_similarity(tfidf_matrix)
    return np.clip((similarities[0][2] - similarities[0][1]) * 50 + 50, 0, 100)

In [ ]:
results_df['negative_rejection_score'] = results_df.apply(
    lambda row: calc_negative_rejection(
        # Extract text from nested document structure
        extract_text_from_nested(row['original_documents_sentences']),
        extract_text_from_nested(row['retrieved_relevant_documents_sentences']),
        extract_text_from_nested(row['noisy_response'])
    ),
    axis=1
)

In [24]:
results_df.to_csv('Rahul RAG llm evaluationmetric results_df.csv', index=False)
results_df

,id,dataset_name,question,original_documents,original_response,retrieved_documents,qwen_05B,DeepSeek,llm_response,original_documents_sentences,...,original_documents_identifiers,retrieved_documents_identifiers,retrieved_relevant_documents_sentences,retrieved_relevant_documents_sentences_identifiers,context_relevance,utilization_score,completeness_score,is_adherent,noisy_response,noise_robustness
0,358,covidqa,What role does T-cell count play in severe hum...,['Title: Emergent severe acute respiratory dis...,The T-cell count plays a crucial role in sever...,['Title: Emergent severe acute respiratory dis...,What role does T-cell count play in severe hum...,Question: What role does T-cell count play in ...,"The semicontaminated area, as mentioned in the...","[[[0a, Title: Emergent severe acute respirator...",...,"[0a, 0b, 0c, 0d, 0e, 0f, 1a, 1b, 1c, 1d, 1e, 1...","[0a, 0b, 0c, 0d, 0e, 0f, 1a, 1b, 1c, 1d, 1e, 1...","[[[0a, Title: Emergent severe acute respirator...","['0a', '0b', '0c', '0d', '0e', '0f', '1a', '1b...",0.526316,0.905835,1.000000,True,"[[[0a, Title: Emergent severe acute respirator...",0.0
1,457,covidqa,What is MVO?,['Title: Gene Knockdowns in Adult Animals: PPM...,MVO is not defined or mentioned in the provide...,['Intraocular pressure (IOP) is the fluid pres...,What is MVO?['Title: Gene Knockdowns in Adult ...,Question: What is MVO?\nContext: ['Title: Gene...,"Based on the context provided, as of 21 Februa...","[[[0a, Title: Gene Knockdowns in Adult Animals...",...,"[0a, 0b, 0c, 1a, 1b, 1c, 1d, 2a, 2b, 2c, 2d, 2...","[0a, 1a, 1b, 1c, 1b, 1c, 1d, 1e, 1f, 1b, 1c, 1...",[],[],0.000000,0.905835,0.000000,True,[],0.0
2,1172,covidqa,What have sero-surveys of MERS virus found?,"['Title: MERS coronavirus: diagnostics, epidem...",Sero-surveys of MERS virus found no sign of ME...,"['Title: MERS coronavirus: diagnostics, epidem...",What have sero-surveys of MERS virus found?['T...,Question: What have sero-surveys of MERS virus...,"Based on the context provided, sero-surveys of...","[[[0a, Title: MERS coronavirus: nogicssdia,t e...",...,"[0a, 0b, 0c, 0d, 0e, 0f, 1a, 1b, 1c, 1d, 1e, 1...","[0a, 0b, 0c, 0d, 0e, 0f, 1a, 1b, 1c, 2a, 2b, 2...","[[[0a, Title: MERS coronavirus: nogicssdia,t e...","['0a', '0b', '0c', '0d', '0e', '0f']",0.260870,0.905835,0.000000,True,"[[[0a, Title: MERS coronavirus: nogicssdia,t e...",0.0
3,1033,covidqa,Where was alpha-tubulin found least abundantly...,['Title: Nucleolar Protein Trafficking in Resp...,Alpha-tubulin was found least abundantly in th...,['Title: Nucleolar Protein Trafficking in Resp...,Where was alpha-tubulin found least abundantly...,Question: Where was alpha-tubulin found least ...,Alpha-tubulin was found to be weakly detected ...,"[[[0a, Title: Nucleolar Protein Trafficking in...",...,"[0a, 0b, 0c, 0d, 0e, 0f, 1a, 1b, 1c, 2a, 2b, 2...","[0a, 0b, 0c, 0d, 0e, 0f, 1a, 1b, 1c, 1d, 1e, 2...","[[[0a, Title: Nucleolar Protein Trafficking in...","['0a', '0b', '0c', '0d', '0e', '0f', '1a', '1b...",0.611111,0.905835,0.250000,False,"[[[0a, Title: Nucleolar Protein Trafficking in...",0.0
4,1273,covidqa,What does the semicontaminated area include?,['Title: The Battle Against Coronavirus Diseas...,The semicontaminated area includes the fever-C...,['Title: The Battle Against Coronavirus Diseas...,What does the semicontaminated area include?['...,Question: What does the semicontaminated area ...,"In the context provided, T-cell count seems to...","[[[0a, Title: The laeBtt Against Coronavirus D...",...,"[0a, 0b, 1a, 1b, 2a, 2b, 2c, 3a, 3b, 3c]","[0a, 0b, 0c, 1a, 1b, 2a, 2b, 2b, 2b, 2c, 2b, 2...","[[[0a, Title: The laeBtt Against Coronavirus D...","['0a', '0b']",0.200000,0.905835,0.250000,True,"[[[0a, Title: The laeBtt Against Coronavirus D...",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,finqa_test_96,tatqa,When are multi-year agreement customers invoiced?,"['Contract Balances [[""(In millions)"", """", """",...",Multi-year agreement customers are gene